In [2]:
import pandas as pd


In [2]:
target_zip_id=197
target_num = int(target_zip_id)
start_range = (target_num // 1000) * 1000
end_range = start_range + 999
start_range = "{:04d}".format(start_range)
end_range = "{:04d}".format(end_range)

# 範囲を文字列で表現
output_range = f"{start_range}-{end_range}"
print(target_num, output_range)

end_range


197 0000-999


999

In [3]:
"""
df=pd.read_csv("data/cc-hosts-20230303.csv")
ja_df=df[df["tld"]=="jp"]
ja_df=ja_df[ja_df["file_name"]==ja_df["file_name"]]
ja_df = ja_df.drop_duplicates(subset=['file_name'])

ja_df.to_csv("data/ja_df.csv")
"""

/tmp/ipykernel_2814574/3724535856.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("data/cc-hosts-20230303.csv")


In [6]:
ja_df=pd.read_csv("data/ja_df.csv")
ja_df

,Unnamed: 0,url_id,file_name,host,tld,ip_address,country,latitude,longitude
0,7,2902384,0000007.pdf,www.acutenet.co.jp,jp,210.248.135.16,JP,35.6897,139.6895
1,8,3821405,0000008.pdf,yokosukaclimatecase.jp,jp,157.112.152.50,JP,35.6897,139.6895
2,25,7021015,0000025.pdf,kanagawa-pho.jp,jp,60.43.148.215,JP,35.6897,139.6895
3,70,4712249,0000068.pdf,www.fireeye.jp,jp,NaN,NaN,NaN,NaN
4,83,2881847,0000081.pdf,a-s-k.co.jp,jp,163.43.87.179,JP,35.6897,139.6895
...,...,...,...,...,...,...,...,...,...
238428,8300038,3120950,7932770.pdf,www.jmac.co.jp,jp,203.137.179.118,JP,35.6897,139.6895
238429,8300062,1890705,7932793.pdf,www.saitamakeikyo.or.jp,jp,18.178.117.43,JP,35.6887,139.7450
238430,8300092,2254820,7932822.pdf,ipsj.ixsq.nii.ac.jp,jp,59.106.216.65,JP,35.6642,140.0475
238431,8300108,3481549,7932835.pdf,nissei-jyuken.co.jp,jp,157.205.173.10,JP,35.6897,139.6895


In [19]:
from collections import Counter
#target_host=".ac"
#target_df=ja_df[ja_df["host"].str.find(target_host)>0]
target_df=ja_df
print(target_df.shape)
file_name_list=list(target_df["file_name"])

zip_index_list=[i[:4] for i in file_name_list]

counts=Counter(zip_index_list)
sorted_counts = counts.most_common()
download_zip_order=[i[0] for i in sorted_counts]
download_zip_order

(238433, 9)


['3336',
 '1329',
 '6570',
 '2868',
 '3099',
 '4557',
 '1138',
 '7413',
 '0198',
 '0894',
 '5586',
 '7070',
 '7747',
 '0062',
 '1463',
 '3136',
 '3983',
 '4879',
 '5450',
 '6072',
 '6326',
 '6471',
 '6663',
 '6808',
 '6845',
 '6889',
 '7420',
 '1207',
 '1574',
 '2347',
 '2390',
 '3767',
 '3842',
 '4088',
 '4490',
 '4532',
 '5029',
 '5147',
 '5204',
 '5505',
 '6154',
 '6355',
 '6779',
 '7140',
 '7791',
 '0496',
 '0844',
 '1505',
 '1761',
 '2411',
 '2743',
 '2778',
 '2992',
 '2998',
 '3020',
 '3146',
 '3571',
 '4422',
 '4740',
 '4808',
 '4923',
 '5078',
 '5196',
 '5416',
 '5529',
 '5946',
 '5961',
 '6071',
 '6076',
 '6079',
 '6890',
 '6910',
 '7321',
 '7425',
 '7604',
 '7708',
 '7778',
 '7883',
 '0070',
 '0472',
 '0845',
 '0905',
 '1087',
 '1262',
 '1381',
 '1434',
 '2183',
 '2407',
 '2410',
 '2480',
 '2544',
 '2763',
 '2797',
 '3037',
 '3323',
 '3344',
 '3920',
 '4239',
 '4248',
 '4301',
 '4407',
 '4435',
 '4572',
 '5066',
 '5701',
 '5711',
 '5721',
 '5729',
 '5924',
 '6268',
 '6375',
 

In [17]:
import requests
from zipfile import ZipFile
import os
import glob
def make_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)


def get_url(target_zip_id):
    base_url="https://digitalcorpora.s3.amazonaws.com/corpora/files/CC-MAIN-2021-31-PDF-UNTRUNCATED/zipfiles"
    # 目標のZIP IDから範囲を生成する
    target_num = int(target_zip_id)
    start_range = (target_num // 1000) * 1000
    end_range = start_range + 999

    # 範囲を文字列で表現
    output_range = f"{start_range}-{end_range}"

    url=f"{base_url}/{output_range}/{target_zip_id}.zip"

    return url

def download_and_extract_zip(target_url, target_dir):
    # ZIPファイルをダウンロードするための一時ファイル名を生成
    temp_zip_path = os.path.join(target_dir, "temp.zip")
    
    # URLからZIPファイルをダウンロード
    response = requests.get(target_url)
    response.raise_for_status()  # エラーが発生した場合は例外を発生させる
    
    # ダウンロードした内容を一時ファイルに保存
    with open(temp_zip_path, 'wb') as temp_zip:
        temp_zip.write(response.content)
    
    # ZIPファイルを解凍
    with ZipFile(temp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    
    # 一時ファイルを削除
    os.remove(temp_zip_path)



def remove_irrelevant_pdf(target_zip_id,file_name_list):
    for pdf_path in glob.glob(f"data/{target_zip_id}/*.pdf"):
        filename=pdf_path.split("/")[-1]
        if filename not in file_name_list:
            os.remove(pdf_path)


def download_pdf(target_zip_id,file_name_list):
    target_dir=f"data/{target_zip_id}"
    if os.path.exists(target_dir):
        print("already finished")
        return
    make_dir(target_dir)
    target_url=get_url(target_zip_id)
    download_and_extract_zip(target_url, target_dir)
    remove_irrelevant_pdf(target_zip_id,file_name_list)

In [18]:
target_zip_id="2928"
for target_zip_id in download_zip_order:
    download_pdf(target_zip_id,file_name_list)

already finished
